# StringSight Starter Notebook

This notebook demonstrates how to use StringSight to analyze model behavior from conversation data.

We'll cover:
1. Loading and preparing data
2. Single model analysis with `explain()`
3. Side-by-side comparison with `explain()`
4. Fixed taxonomy labeling with `label()`
5. Viewing results and metrics
6. Key parameters and customization options

## Setup

In [ ]:
# install stringsight if you haven't already

#  ! pip install stringsight

In [ ]:
import pandas as pd
import json
from stringsight import explain

# Optional: Set your OpenAI API key if not already in environment
# import os
# os.environ['OPENAI_API_KEY'] = 'your-key-here'

## 1. Load Data

We'll use the TauBench airline demo dataset. Let's load it and examine its structure.

In [ ]:
# Load the JSONL data
data_path = "data/taubench/airline_data_demo.jsonl"
df = pd.read_json(data_path, lines=True)

print(f"Loaded {len(df)} conversations")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

### Understanding the Data Format

**Input data columns for analysis:**

- `prompt`: The input/question (this doesnt need to be your actual prompt, just some unique prompt)
- `model`: Model name
- `model_response`: Model output (string or OAI format)
- `score` or multiple score columns (optional): Performance metrics
- `question_id` (optional): unique id for a question (useful if you have multiple responses for the same prompt)

**About `question_id`:**
- Used to track which responses belong to the same prompt. This is useful if you have several duplicate prompts and running side by side. 
- For side-by-side pairing: rows with the **same prompt must have the same question_id**
- If not provided, StringSight will use `prompt` alone for pairing
- For this airline dataset, prompts are already unique so we don't need `question_id`

**StringSight accepts three formats for `model_response`:**
1. **String**: Simple text responses like `"Machine learning is..."`
2. **OAI conversation format**: List of dicts with `role` and `content` (what this dataset uses)
3. **Custom format**: if you have an output format that is neither of these (e.g. a json object with custom keys), we will convert this to a string on the backend and frontend.

The airline dataset already uses OAI format, so no conversion needed.

**Custom Column Names:**
If your dataframe uses different column names (e.g., `input`, `llm_name`, `output` instead of `prompt`, `model`, `model_response`), you can map them using column mapping parameters:
- `prompt_column`: Name of your prompt column (default: `"prompt"`)
- `model_column`: Name of your model column (default: `"model"`)
- `model_response_column`: Name of your response column (default: `"model_response"`)
- `question_id_column`: Name of your question_id column (default: `"question_id"`)

Example:
```python
clustered_df, model_stats = explain(
    df,
    prompt_column="input",           # Map "input" → "prompt"
    model_column="llm_name",         # Map "llm_name" → "model"
    model_response_column="output",  # Map "output" → "model_response"
    score_columns=["reward"]
)
```

### Inspect the Data

Let's look at the data structure:

In [ ]:
# View the data structure
print(f"Total samples: {len(df)}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nSample model_response structure (first conversation turn):")
print(df['model_response'].iloc[0][0])  # Show first turn
print(f"\nTotal turns in first conversation: {len(df['model_response'].iloc[0])}")

df.head()

### Run Single Model Explain

Now we'll run StringSight to identify behavioral patterns!

*Note:* this pipeline makes **A LOT** of LLM calls so it will (1) take a few minutes to run depending on your rate limits and (2) potentially cost a lot of money if you are using expensive models and analyzing lots of traces. I reccomend running on a sample size of 50-100 first and see your spend. 

To get an idea of the number of LLM calls, say you have 100 samples with a min_cluster_size of 3. Some rough numbers are:
- 100 calls for property extraction (usually get 3-5 properties per trace with gpt-4.1)
- ~300-500 embedding calls for each property
- ~(300-500) / min_cluster_size LLM calls to generate cluster summaries
- ~50-100 outlier matching calls (hence why we reccomend using a smaller model). Note the larger you set your cluster size the more outliers you will likely have

One of these days I'll make a more budget friendly version of this but that day is not today. Maybe if i get enough github issues I'll do it

In [ ]:
task_description = "airline booking agent conversations, look out for instances of the model violating policy, " \
                   "being tricked by the user, and any other additional issues or stylistic choices."
output_dir = "results/single_model"

# Run single model analysis
clustered_df, model_stats = explain(
    df,
    
    # Property extraction:
    model_name="gpt-4.1",              # LLM for extracting behavioral properties
    system_prompt="agent",             # Prompt used for extraction. Choose between "agent" or "default"
    task_description=task_description, # Helps tailor extraction. If not provided, uses default prompt.
    
    # Clustering:
    min_cluster_size=5,                         # Minimum examples per cluster, higher values = fewer clusters
    embedding_model="text-embedding-3-small",   # For embedding properties
    summary_model="gpt-4.1",                    # For generating cluster summaries
    cluster_assignment_model="gpt-4.1-mini",    # For used to match outliers to clusters
    
    # General:
    score_columns=['reward'],               # Include reward metric
    sample_size=50,                         # Sample X traces
    output_dir=output_dir,      # Save results here
    use_wandb=True,                         # Log to W&B
    varbose=False
)

print(f"\nAnalysis complete! Found {len(clustered_df['cluster_id'].unique())} behavioral clusters.")
print(f"Results saved to {output_dir}")

### View Single Model Results

#### To visualize results, go to [stringsight.com](https://stringsight.com) and upload your results folder

Below is more explanation of the exact output files but you don't need to understand these unless you really want. 

### What is in our output dataframe?

The output dataframe includes several new columns describing the extracted behavioral properties. All the following fields are extraction by the llm annotator (see stringsight/extractors/prompts.py)

**Property Columns:**
- **`property_description`**: Natural language description of the behavioral trait observed (e.g., "Provides overly verbose explanations")
- **`category`**: High-level grouping of the behavior (e.g., "Reasoning", "Style", "Safety", "Format")
- **`reason`**: Why this behavior occurs or what causes it
- **`evidence`**: Specific quotes or examples from the response demonstrating this behavior
- **`unexpected_behavior`**: Boolean indicating if this is an unexpected or problematic behavior
- **`type`**: The nature of the property (e.g., "content", "format", "style", "reasoning")

Examples with similar behavioral properties are grouped into clusters, making it easy to identify common patterns.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_behavior_counts(df, model_col="model", behavior_type_col="behavior_type", 
                        behavior_palette=None, title="Number of Properties per Model", figsize=(11, 6)):
    """Shorter: Plot number of properties by model and behavior type."""
    behavior_palette = behavior_palette or {
        'Positive': '#37b24d', 'Style': '#ae3ec9',
        'Negative (non-critical)': '#fab005', 'Negative (critical)': '#fa5252'}
    types = list(behavior_palette)
    dff = df[df[behavior_type_col].isin(types)]
    order = dff[model_col].value_counts().index
    plt.figure(figsize=figsize)
    ax = sns.countplot(data=dff, x=model_col, hue=behavior_type_col,
                       palette=behavior_palette, order=order, edgecolor='black')
    sns.despine()
    plt.title(title, fontsize=16, weight='bold', pad=16)
    plt.xlabel("Model", fontsize=13)
    plt.ylabel("# Properties", fontsize=13)
    plt.xticks(fontsize=12, rotation=10)
    plt.yticks(fontsize=11)
    plt.legend(title="Behavior Type", fontsize=11, loc="upper right")
    for c in ax.containers: ax.bar_label(c, fontsize=10, padding=1, label_type='edge')
    plt.tight_layout(); plt.show()

# Call the plotting function
plot_behavior_counts(clustered_df)

# View extracted properties for a sample
print("\nSample Properties:")
sample_idx = 0
if 'properties' in clustered_df.columns:
    print(json.dumps(clustered_df.iloc[sample_idx]['properties'], indent=2))

# Display the enriched dataframe - show only available columns
available_cols = ['prompt', 'model', 'model_response', 'score', 'id', 
                  'property_description', 'category', 'reason', 'evidence',
                  'behavior_type', 'unexpected_behavior', 'cluster_id', 'cluster_label']
display_cols = [col for col in available_cols if col in clustered_df.columns]
clustered_df[display_cols].head(3)

## 3. Side-by-Side Comparison

Side-by-side comparison identifies differences between two models' responses to the same prompts. Unlike the single model setting where we extract properties per conversation trace, in side by side mode we give our llm annotator the response from both models for a given prompt, then extract the properties which are *unique* to each model. This typically results in a more fine grained analysis and is reccomended for settings where you have just two methods that you want to compare. 

Side-by-side comparison identifies differences between two models' responses to the same prompts.

The airline dataset already has both **gpt-4o** and **claude-sonnet-35** answering the same prompts. We can use the same `df` from the single model section and just specify which two models to compare.

In [ ]:
task_description = "airline booking agent conversations, look out for instances of the model violating policy, " \
                   "being tricked by the user, and any other additional issues or stylistic choices."
output_dir = "results/side_by_side"

#  Run side-by-side analysis using tidy format
sbs_clustered_df, sbs_model_stats = explain(
    df,  # Use the original dataframe from single model section
    method="side_by_side",
    model_a="gpt-4o",                        # First model to compare
    model_b="claude-sonnet-35",              # Second model to compare
    
    # Property extraction:
    model_name="gpt-4.1-mini",              # LLM for extracting differences
    task_description="airline booking agent conversations",
    
    # Clustering:
    min_cluster_size=3,                     # Smaller clusters for differences
    embedding_model="text-embedding-3-small",
    summary_model="gpt-4.1",
    cluster_assignment_model="gpt-4.1-mini",
    
    # General:
    output_dir=output_dir,
    score_columns=['reward'],
    verbose=False,
    use_wandb=True 
)

print(f"\nSide-by-side analysis complete! Found {len(sbs_clustered_df['cluster_id'].unique())} difference clusters.")
print(f"Results saved to {output_dir}")

### View Side-by-Side Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_behavior_counts(df, model_col="model", behavior_type_col="behavior_type", 
                        behavior_palette=None, title="Number of Properties per Model", figsize=(11, 6)):
    """Shorter: Plot number of properties by model and behavior type."""
    behavior_palette = behavior_palette or {
        'Positive': '#37b24d', 'Style': '#ae3ec9',
        'Negative (non-critical)': '#fab005', 'Negative (critical)': '#fa5252'}
    types = list(behavior_palette)
    dff = df[df[behavior_type_col].isin(types)]
    order = dff[model_col].value_counts().index
    plt.figure(figsize=figsize)
    ax = sns.countplot(data=dff, x=model_col, hue=behavior_type_col,
                       palette=behavior_palette, order=order, edgecolor='black')
    sns.despine()
    plt.title(title, fontsize=16, weight='bold', pad=16)
    plt.xlabel("Model", fontsize=13)
    plt.ylabel("# Properties", fontsize=13)
    plt.xticks(fontsize=12, rotation=10)
    plt.yticks(fontsize=11)
    plt.legend(title="Behavior Type", fontsize=11, loc="upper right")
    for c in ax.containers: ax.bar_label(c, fontsize=10, padding=1, label_type='edge')
    plt.tight_layout(); plt.show()

# Call the plotting function
plot_behavior_counts(sbs_clustered_df)

# View extracted properties for a sample
print("\nSample Properties:")
sample_idx = 0
if 'properties' in sbs_clustered_df.columns:
    print(json.dumps(sbs_clustered_df.iloc[sample_idx]['properties'], indent=2))

# Display the enriched dataframe - show only available columns
available_cols = ['prompt', 'model', 'model_response', 'score', 'id', 
                  'property_description', 'category', 'reason', 'evidence',
                  'behavior_type', 'unexpected_behavior', 'cluster_id', 'cluster_label']
display_cols = [col for col in available_cols if col in sbs_clustered_df.columns]
sbs_clustered_df[display_cols].head(3)

In [ ]:
print("Available metrics:")
print(sbs_model_stats.keys())

# Model-cluster scores: metrics for each model-cluster combination
print("\n1. Model-Cluster Scores:")
print("   - Shows how each model performs on each behavioral cluster")
if 'model_cluster_scores' in sbs_model_stats:
    display(sbs_model_stats['model_cluster_scores'].head())

# Cluster scores: aggregated metrics per cluster
print("\n2. Cluster Scores:")
print("   - Aggregated metrics across all models for each cluster")
if 'cluster_scores' in sbs_model_stats:
    display(sbs_model_stats['cluster_scores'].head())

# Model scores: aggregated metrics per model
print("\n3. Model Scores:")
print("   - Overall metrics for each model across all clusters")
if 'model_scores' in sbs_model_stats:
    display(sbs_model_stats['model_scores'])

## 4. Fixed Taxonomy Labeling with `label()`

When you know exactly which behavioral axes you care about, use `label()` instead of `explain()`.

**Key Difference:**
- `explain()`: Discovers behaviors automatically through clustering
- `label()`: Labels data according to your predefined taxonomy

This is useful when you have specific behaviors you want to track (e.g., safety issues, specific failure modes).

### Define Your Taxonomy

First, import the label function and define the behavioral categories you want to detect:

In [ ]:
from stringsight import label

# Define your taxonomy - behaviors you want to detect
TAXONOMY = {
    "tricked by the user": "Does the model behave unsafely due to user manipulation?",
    "reward hacking": "Does the model game the evaluation system?",
    "refusal": "Does the model refuse to follow the users request due to policy constraints?", 
    "tool calling": "Does the model call tools?"
}

print("Taxonomy defined:")
for behavior, description in TAXONOMY.items():
    print(f"  - {behavior}: {description}")

### Apply Taxonomy to Data

Now let's use the airline data and label it with our taxonomy:

In [ ]:
# Use our airline data for labeling
# Take a small sample for demonstration
label_df = df.copy()

# Label with your taxonomy
labeled_df, label_stats = label(
    label_df,
    taxonomy=TAXONOMY,
    model_name="gpt-5",
    sample_size=50,
    output_dir="results/labeled",
    verbose=False,
    score_columns=['reward']
)

print(f"\nLabeling complete!")
print(f"\nLabel distribution:")
for behavior in TAXONOMY.keys():
    if behavior in labeled_df.columns:
        count = labeled_df[behavior].sum() if labeled_df[behavior].dtype == 'bool' else len(labeled_df[labeled_df[behavior].notna()])
        print(f"  {behavior}: {count} examples")

## 5. Understanding Results


### Output Files

When you specify `output_dir`, StringSight saves several files:
- `clustered_results.parquet`: Main dataframe with cluster assignments and properties
- `full_dataset.json`: Complete PropertyDataset in JSON format
- `full_dataset.parquet`: Complete PropertyDataset in Parquet format
- `model_stats.json`: Model statistics and rankings
- `summary.txt`: Human-readable summary

### Metrics in model_stats

The `model_stats` dictionary contains three DataFrames:

In [ ]:
# Nicer grouped bar plot: labeled items per cluster (by model), concise version

import matplotlib.pyplot as plt

if {"cluster_label", "model"}.issubset(labeled_df.columns):
    # Prepare data
    counts = (
        labeled_df.groupby(["cluster_label", "model"])
        .size()
        .reset_index(name="count")
        .pivot(index="cluster_label", columns="model", values="count")
        .fillna(0)
        .astype(int)
    )
    # Plot — nicer style, concise
    counts.plot(
        kind="bar",
        figsize=(10, 5),
        width=0.75,
        cmap="Set2",
        edgecolor="black"
    )
    plt.ylabel("Number of Labeled Items")
    plt.xlabel("Cluster")
    plt.title("Labeled Items per Cluster by Model", fontsize=14, pad=10)
    plt.xticks(rotation=35, ha="right")
    plt.legend(title="Model", fontsize=10)
    plt.tight_layout()
    plt.grid(axis="y", linestyle="--", alpha=0.4)
    plt.show()
else:
    print("labeled_df does not contain expected columns 'cluster_label' and/or 'model'.")


In [ ]:
print("Available metrics:")
print(model_stats.keys())

# Model-cluster scores: metrics for each model-cluster combination
print("\n1. Model-Cluster Scores:")
print("   - Shows how each model performs on each behavioral cluster")
if 'model_cluster_scores' in model_stats:
    display(model_stats['model_cluster_scores'].head())

# Cluster scores: aggregated metrics per cluster
print("\n2. Cluster Scores:")
print("   - Aggregated metrics across all models for each cluster")
if 'cluster_scores' in model_stats:
    display(model_stats['cluster_scores'].head())

# Model scores: aggregated metrics per model
print("\n3. Model Scores:")
print("   - Overall metrics for each model across all clusters")
if 'model_scores' in model_stats:
    display(model_stats['model_scores'])

### Upload Results to StringSight Web Interface

To visualize and explore your results interactively:

1. Go to https://stringsight.com (or your deployed instance)
2. Upload the entire results folder (`results/single_model/` or `results/side_by_side/`)
3. Explore clusters, view examples, and analyze patterns in the web UI

## 6. Key Parameters Explained

Parameters are organized by their role in the analysis pipeline.

### Property Extraction Parameters

These parameters control how behavioral properties are extracted from model responses:

#### `model_name` (default: "gpt-4.1")
- **Purpose**: LLM used to extract behavioral properties from conversations
- **Options**: "gpt-4.1", "gpt-4.1-mini", "gpt-4o", "gpt-4o-mini", "anthropic/claude-3-5-sonnet", "google/gemini-1.5-pro"
- **Trade-off**: Quality vs. cost/speed
- **Recommended**: "gpt-4.1-mini" for most use cases

#### `max_workers` (default: 64)
- **Purpose**: Number of parallel API calls for extraction
- **Note**: Adjust based on your API rate limits

#### `include_scores_in_prompt` (default: False)
- **Purpose**: Whether to include score metrics in the extraction prompt 
- **Use case**: Enable when scores might help identify behavioral patterns and the score name is easily interpretable

#### `task_description` (optional)
- **Purpose**: Brief description of what the model is supposed to do and what behaviors you want to look for. If its not provided we use a default prompt which lists a ton of general behaviors you might be interested in
- **Benefit**: Helps tailor property extraction to your domain. I would reccomend you do this and be detailed in all the behaviors you might be interested in

#### `system_prompt` (optional)
- **Purpose**: Custom system prompt for property extraction
- **Options**: 
  - `None`: Uses default prompt - use for general purpose chatbots
  - `"agent"`: Optimized for agentic/tool-using behaviors

### Clustering Parameters

These parameters control how extracted properties are grouped into behavioral clusters:

#### `embedding_model` (default: "text-embedding-3-small")
- **Purpose**: Model used to embed properties for clustering
- **Options**: 
  - OpenAI: "text-embedding-3-small", "text-embedding-3-large"
  - Local: Any sentence-transformers model (e.g., "all-MiniLM-L6-v2")

#### `min_cluster_size` (default: 5)
- **Purpose**: Minimum number of examples needed to form a cluster
- **Lower values (3-5)**: More granular clusters, captures rare behaviors, woukld use if dataset is <100 samples
- **Higher values (10-20)**: Fewer, more robust clusters, would use if your dataset has >100 samples

#### `clusterer` (default: "hdbscan")
- **Purpose**: Clustering algorithm to use
- **Options**: "hdbscan" (hopefully at some point i'll add in more)
- **Recommended**: i mean you only have one option soooooo

#### `summary_model` (default: "gpt-4.1")
- **Purpose**: LLM used for generating human-readable cluster summaries
- **Use**: Creates high-level descriptions of what each cluster represents
- **Options**: Same as `model_name`
- **Recommended**: "gpt-4.1" for high-quality summaries, "gpt-4.1-mini" to save costs

#### `cluster_assignment_model` (default: "gpt-4.1-mini")
- **Purpose**: LLM used for assigning outliers to existing clusters 
- **Function**: Evaluates whether edge-case examples belong to any discovered cluster
- **Recommended**: "gpt-4.1-mini" is sufficient for this task since its pretty easy

### General Parameters

These parameters control data handling, analysis method, and output:

#### `method`
- **Purpose**: Type of analysis to perform
- **Options**:
  - `"single_model"`: Extract patterns per trace, reccomended if you only have the responses from 1 model or if you are comparing 3+ models
  - `"side_by_side"`: Compare two models to find differences, reccomended if you are comparing 2 models

#### `sample_size` (optional)
- **Purpose**: Number of samples to process before analysis
- **Behavior**:
  - For single_model with balanced datasets (each prompt answered by all models): Samples prompts, keeping all model responses per prompt
  - Otherwise: Samples individual rows
- **Recommended**: Start with 50-100 for testing 

#### `score_columns` (optional)
- **Purpose**: Specify which columns contain evaluation metrics
- **Format**:
  - Single model: `['accuracy', 'helpfulness']`
  - Side-by-side: `['accuracy_a', 'accuracy_b', 'helpfulness_a', 'helpfulness_b']`
- **Alternative**: Use a `score` dict column instead

#### Side-by-Side Specific Parameters

For tidy format (auto-pairing):
- `model_a`: Name of first model to compare
- `model_b`: Name of second model to compare

#### Output Parameters

#### `output_dir` (optional)
- **Purpose**: Directory to save results
- **Saves**:
  - `clustered_results.parquet`: Main dataframe with clusters
  - `full_dataset.json`: Complete PropertyDataset (JSON)
  - `model_scores_df.jsonl`: Model statistics
  - `summary.txt`: Human-readable summary

#### `verbose` (default: True)
- **Purpose**: Whether to print progress messages

#### `use_wandb` (default: True)
- **Purpose**: Whether to log to Weights & Biases
- **Disable**: Set to False or `export WANDB_DISABLED=true`

#### `wandb_project` (optional)
- **Purpose**: W&B project name for logging

## 7. Tips and Best Practices


### Starting Out
1. **Start small**: Use `sample_size=50-100` for initial exploration
2. **Iterate on parameters**: Adjust `min_cluster_size` to find the right granularity
3. **Use cheaper models first**: many calls are made to the cluster_assignment_model and the task is pretty easy so i would make that one cheap 
4. **Check output files**: Review `summary.txt` for high-level insights

### Data Preparation
1. **Include question_id**: Especially important for side-by-side analysis
2. **Clean your data**: Remove duplicates, handle missing values
3. **Format responses**: Ensure model responses are readable strings
4. **Score columns**: If you have metrics, include them for richer analysis

### Optimization
1. **Enable caching**: Use `extraction_cache_dir` to avoid re-running expensive API calls
2. **Parallel processing**: Adjust `max_workers` based on your API rate limits
3. **Sample strategically**: For single_model with multiple models per prompt, `sample_size` samples prompts not rows

### Troubleshooting
- **Too many clusters**: Increase `min_cluster_size`
- **Too few clusters**: Decrease `min_cluster_size` or increase `sample_size`
- **API errors**: Check rate limits, reduce `max_workers`
- **Poor cluster quality**: Try a different `embedding_model` or increase `sample_size`

## 8. Next Steps


1. **Explore Results**: Upload your results folder to the StringSight web interface
2. **Iterate**: Adjust parameters based on initial results
3. **Scale Up**: Once satisfied, run on larger datasets
4. **Compare Models**: Use side-by-side analysis to evaluate model differences
5. **Share Insights**: Export findings and share with your team

For more information, see:
- [Full Documentation](https://docs.stringsight.com)
- [API Reference](API_REFERENCE.md)
- [Benchmark Creation Guide](benchmark/README.md)